In [1]:
# load environment variables from .env file (requires `python-dotenv`)
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Translate the following from English into {language}"),
    ("user", "{text}")
])


llm = ChatOpenAI(
    model="llama-3.1-8b-instant",
    base_url="https://api.groq.com/openai/v1",
)

parser = StrOutputParser()

chain = prompt_template | llm | parser

print(chain.invoke({"text": "nice to meet you", "language": "Chinese"}))



很高興見到您


In [3]:
analysis_prompt = ChatPromptTemplate.from_template("我应该怎么回答这句话？{talk}。给我一个五个字以内的示例")

chain2 = {"talk": chain} | analysis_prompt | llm | parser

print(chain2.invoke({"text": "nice to meet you", "language": "Chinese"}))

你好，很高兴见到你。

("nǐ hǎo, hěn gāo xìng jiàn dào nǐ.")


In [4]:
from langchain_core.runnables import RunnableMap, RunnableLambda

prompt_template_zh = ChatPromptTemplate.from_messages([
    ("system", "Translate the following from English into Chinese"),
    ("user", "{text}")
])

prompt_template_fr = ChatPromptTemplate.from_messages([
    ("system", "Translate the following from English into French"),
    ("user", "{text}")
])

# 构建链
chain_zh = prompt_template_zh | llm | parser
chain_fr = prompt_template_fr | llm | parser

# 并行执行两个链
parallel_chains = RunnableMap({
    "zh_translation": chain_zh,
    "fr_translation": chain_fr,
})

# 合并结果
final_chain = parallel_chains | RunnableLambda(lambda x: f"Chinese: {x['zh_translation']}\nFrench: {x['fr_translation']}")

print(final_chain.invoke({"text": "nice to meet you"}))

Chinese: 中文翻译为:

很高兴见到你
French: Enchanté(e).


In [5]:
final_chain.get_graph().print_ascii()

   +----------------------------------------------+    
   | Parallel<zh_translation,fr_translation>Input |    
   +----------------------------------------------+    
                  ***             ***                  
                **                   **                
              **                       **              
+--------------------+          +--------------------+ 
| ChatPromptTemplate |          | ChatPromptTemplate | 
+--------------------+          +--------------------+ 
           *                               *           
           *                               *           
           *                               *           
    +------------+                  +------------+     
    | ChatOpenAI |                  | ChatOpenAI |     
    +------------+                  +------------+     
           *                               *           
           *                               *           
           *                               *    

## 保存聊天记录

In [6]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

history.add_user_message("你是谁？")
ai_message = llm.invoke(history.messages)
print(ai_message.content)
history.add_message(ai_message)

history.add_user_message("请重复一次")
ai_message2 = llm.invoke(history.messages)
print(ai_message2.content)
history.add_message(ai_message2)

我是 LLaMA，一个人工智能语言模型。我的任务是与用户交谈，回答他们的问题，并提供信息，帮助他们解决问题。
我是 LLaMA，一个基于语言的人工智能模型。我的目的在于与用户进行交谈，回答他们的疑问并提供有用的信息。


In [7]:
from langchain_redis import RedisChatMessageHistory

history = RedisChatMessageHistory(session_id="test", redis_url="redis://localhost:6379/0")

# history.add_user_message("你是谁？")
# ai_message = llm.invoke(history.messages)
# print(ai_message.content)
# history.add_message(ai_message)

history.add_user_message("请重复一次")
ai_message2 = llm.invoke(history.messages)
print(ai_message2.content)
history.add_message(ai_message2)

14:40:53 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
您想让我重复什么？


## 3、聊天消息整合 LCEL

In [8]:
from langchain_core.runnables.history import RunnableWithMessageHistory

runnable = RunnableWithMessageHistory(
    llm,
    get_session_history=lambda: history,
)

history.clear()

runnable.invoke({"text": "你是谁？"})
runnable.invoke({"text": "请重复一次"})

14:40:54 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
14:40:54 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='我是 LLaMA (Large Language Model Application)，是一种人工智能语言模型。我的任务是帮助用户回答问题，提供信息，并进行互动对话。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 88, 'total_tokens': 125, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.195844822, 'prompt_time': 0.005135643, 'completion_time': 0.049333333, 'total_time': 0.054468976}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_0f5c9bc037', 'id': 'chatcmpl-367a3bb1-334b-48e3-a0c3-f3c3317f0af8', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--57039b9a-a31e-42b0-a856-d5d37e03e5f4-0', usage_metadata={'input_tokens': 88, 'output_tokens': 37, 'total_tokens': 125, 'input_token_details': {}, 'output_token_details': {}})